<a href="https://colab.research.google.com/github/hallockh/neur_265/blob/main/03_29_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cross-Frequency Coupling

Phase coherence is one way of examining how two simultaneously-recorded signals might interact with each other. Specifically, phase coherence assesses the degree to which two oscillations *within the same frequency band* might co-vary over time. We saw, however, when using the Fourier transform to de-construct our complex signals, that real LFP and EEG signals contain oscillations in many different frequency bands. Do these different frequency bands ever interact with each other?

##By the end of this notebook, you will be able to:

- Extract the phase of an oscillation using the **Hilbert transform**
- Extract the amplitude of another oscillation using the same method
- Calculate the relationship between phase and amplitude
- Plot your data

##Background

In this notebook, we focus on local field potential (LFP) recordings. The LFP is a measure of local population neural activity, produced from small aggregates of neurons. In these data, we examine the association between rhythms of different frequencies.

In general, lower-frequency rhythms have been observed to engage larger brain areas and modulate spatially localized fast oscillations [(see, for example)]('https://pubmed.ncbi.nlm.nih.gov/18388295/'). Of the different types of cross-frequency coupling (CFC) observced between brain rhythms, perhaps the most is phase-amplitude coupling (PAC), in which the phase of a low frequency rhythm modulates the amplitude (or power) of a high frequency rhythm. Cross-frequency coupling been observed in many brain regions, has been shown to change in time with task demands, and has been proposed to serve a [functional role]('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3359652/') in working memory, neuronal computation, communication, and learning. Although the cellular and dynamic mechanisms of specific rhythms associated with CFC are relatively well understood, the mechanisms governing interactions between different frequency rhythms - and the appropriate techniques for measuring CFC - remain active research areas. Although we consider only a single electrode recording here, we note that these techniques can be extended to association measures between electrodes as well.



##Case Study Data

The LFP data we'll be using were recorded from electrodes in the rat hippocampus. These electrodes remain (chronically) implanted as the rat explores a large circular arena. We are interested in assessing the association between different frequency rhythms of the LFP, and more specifically whether an association between different frequency rhythms exists as the rat explores the arena. To address this question, we have 100 s of LFP data recorded during the experiment (i.e., while the rat spontaneously explored the arena).



In [14]:
# Load your modules

import numpy as np
import matplotlib.pyplot as plt

><b>Task:</b> Import the <code>rat_cfc.csv</code> file from our class GitHub repository as a <code>numpy</code> array called <code>rat_cfc</code>.

In [18]:
# Your code here!


The size of our <code>rat_cfc.csv</code> file was too big to be stored as one row, so we needed to split it up into 100 rows. Now that we have it loaded into Colab, let's put it back into a 1-dimensional vector with the <code>numpy</code> method <code>reshape</code>:

In [19]:
# Re-shaping our 100 x 1000 matrix into a 1 x 100000 array

rat_cfc_reshaped = np.reshape(rat_cfc, 100000)

##Data Visualization

Let’s begin with visual inspection of the LFP data. To do that, we'll first need to create a <code>time</code> variable, starting with a value of "0", ending with a value of "100" (recall that this is the length of data, in seconds), with 100000 equally spaced values in between. Do this in the code cell below.

In [20]:
# Make your time variable!


><b>Task:</b> In the code cell below, write an expression that returns the sampling rate of your LFP data (hint - use the fact that you have 100 seconds of data, and 100000 samples).

In [ ]:
# Your code here

><b>Task:</b> Try plotting your data below. Put time on the x-axis, and make your LFP data <font color = 'red'>red</font>. Give your plot some labels and a title!

In [ ]:
# Plot your LFP data!


It's hard to see anything with so much data, so let's zoom in on just a 1 s. interval:

In [ ]:
plt.plot(time, rat_cfc_reshaped)            # Plot the LFP data,
plt.xlim([4, 5])            # ... restrict the x-axis to a 1 s interval,
plt.ylim([-2, 2])           # ... and tighten the y-axis.
plt.xlabel('Time [s]')      # Label the axes
plt.ylabel('Voltage [mV]')

Question: What do you notice about the data? What is the dominant oscillation? What about other oscillations? Any relationship between the two?

##Spectral Analysis

Visual inspection of the LFP data suggests that multiple rhythms appear. To further characterize this observation, we compute the spectrum of the LFP data. We analyze the entire 100 s of data and compute the spectrum with a Hanning taper.

In [ ]:
dt = time[1] - time[0]                # Define the sampling interval,
T = time[-1]                       # ... the duration of the data,
N = len(rat_cfc_reshaped)                    # ... and the no. of data points

x = np.hanning(N) * rat_cfc_reshaped            # Multiply data by a Hanning taper
xf = np.fft.rfft(x - x.mean())         # Compute Fourier transform
Sxx = 2*dt**2/T * (xf*np.conj(xf)) # Compute the spectrum
Sxx = np.real(Sxx)                 # Ignore complex components

df = 1 / T                      # Define frequency resolution,
fNQ = 1 / dt / 2                # ... and Nyquist frequency. 

faxis = np.arange(0, fNQ + df, df) # Construct freq. axis
plt.plot(faxis, 10 * np.log10(Sxx))    # Plot spectrum vs freq.
plt.xlim([0, 200])                  # Set freq. range, 
plt.ylim([-80, 0])                  # ... and decibel range
plt.xlabel('Frequency [Hz]')        # Label the axes
plt.ylabel('Power [mV$^2$/Hz]');

The resulting spectrum reveals two intervals of increased power spectral density. The lowest-frequency peak at 6 Hz is also the largest and corresponds to the slow rhythm we observe dominating the signal through visual inspection. Remember a plot of that signal: At higher frequencies, we find an additional broadband peak at approximately 80–120 Hz. These spectral results support our initial visual inspection of the signal; there exist both low- and high-frequency activities in the LFP data. We now consider the primary question of interest: Do these different frequency rhythms exhibit associations?


##Phase-Amplitude Coupling

To assess whether different frequency rhythms interact in the LFP recording, we implement a measure to calculate the CFC. The idea of CFC analysis, in our case, is to determine whether a relation exists between the phase of a low-frequency signal and the envelope or amplitude of a high-frequency signal. In general, computing CFC involves three steps. Each step contains important questions and encompasses entire fields of study. Our goal in this section is to move quickly forward and produce a procedure we can employ, investigate, and criticize. Continued study of CFC - and the associated nuances of each step - is an active area of ongoing [research]('https://pubmed.ncbi.nlm.nih.gov/26549886/').

##CFC Analysis Steps

- Filter the data into high- and low-frequency bands.

- Extract the amplitude and phase from the filtered signals.

- Determine if the phase and amplitude are related.

The first step in the CFC analysis is to filter the data into two frequency bands of interest. The choice is not arbitrary: the separate frequency bands are motivated by initial spectral analysis of the LFP data. In this case, we choose the low-frequency band as 5–7 Hz, consistent with the largest peak in the spectrum, and the high-frequency band as 80–120 Hz, consistent with the second-largest broadband peak. To consider alternative frequency bands, the same analysis steps would apply.

There are many options to perform the filtering. To do so requires us to design a filter that ideally extracts the frequency bands of interest without distorting the results. Here, we apply a finite impulse response (FIR) filter. We start by extracting the low-frequency band:

In [24]:
from scipy import signal
Wn = [5,7];                         # Set the passband [5-7] Hz,
n = 100;                            # ... and filter order,
                                    # ... build the bandpass filter,
b = signal.firwin(n, Wn, fs=fNQ, pass_zero=False, window='hamming');
Vlo = signal.filtfilt(b, 1, rat_cfc_reshaped);

Next, we extract the high-frequency band:

In [26]:
Wn = [80, 120];                     # Set the passband [80-120] Hz,
n = 100;                            # ... and filter order,
                                    # ... build the bandpass filter,
b = signal.firwin(n, Wn, fs=fNQ, pass_zero=False, window='hamming');
Vhi = signal.filtfilt(b, 1, rat_cfc_reshaped);   # ... and apply it to the data.

For each frequency band, we specify a frequency interval of interest by defining the low- and high-cutoff frequencies in the variable `Wn`. In this way, we specify the passband of the filter. We then set the filter order (`n`) and design the filter using the Python function `signal.firwin` from the SciPy package. Finally, we apply the filter using the Python function `signal.filtfilt`, which performs zero-phase filtering by applying the filter in both the forward and reverse directions. We note that, in this case, the filtering procedure is nearly the same in both frequency bands; the only change is the specification of the frequency interval of interest.

To understand the impact of this filtering operation on the LFP, let’s plot the results. More specifically, let’s plot the original signal, and the signal filtered in the low- and high-frequency bands, for an example 2 s interval of time:

In [ ]:
plt.plot(time, rat_cfc_reshaped)                    # Plot the original data vs time.
plt.plot(time, Vlo)                    # Plot the low-frequency filtered data vs time.
plt.plot(time, Vhi)                    # Plot the high-frequency filtered data vs time.
plt.xlabel('Time [s]')
plt.xlim([24, 26]);                 # Choose a 2 s interval to examine
plt.ylim([-2, 2]);
plt.legend(['LFP', 'Vlo', 'Vhi']);  # Add a legend.

As expected, the low-frequency band (orange) captures the large-amplitude rhythm dominating the LFP signal, while the higher-frequency band (green) isolates the brief bursts of faster activity.


The next step in the CFC procedure is to extract the phase of the low-frequency signal and the amplitude envelope (or simply, amplitude) of the high-frequency signal. To compute a particular type of CFC, namely phase-amplitude coupling, we then compare these two signals, i.e., we compare the phase of the low-frequency activity and the amplitude envelope of the high-frequency activity. How do we actually extract the phase and amplitude signals from the data? There are a variety of options to do so, and we choose here to employ the analytic signal approach, which allows us to estimate the instantaneous phase and amplitude envelope of the LFP.

The first step in computing the analytic signal is to compute the Hilbert transform. We begin with some notation. Define 𝑥
 as a narrowband signal (i.e., a signal with most of its energy concentrated in a narrow frequency range note, e.g., the low- or high-frequency band filtered signals we’ve created). Then the Hilbert transform of 𝑥
, let’s call it 𝑦
, is

𝑦=𝐻(𝑥).


It’s relatively easy to compute the analytic signal and extract the phase and amplitude in Python:

In [ ]:
phi = np.angle(signal.hilbert(Vlo))  # Compute phase of low-freq signal
amp = abs(signal.hilbert(Vhi))       # Compute amplitude of high-freq signal

Question: Why do we use the <code>abs</code> function (absolute value) to return the amplitude of our high-frequency signal?

As with the previous steps, we have at our disposal a variety of procedures to assess the relation between the phase (of the low-frequency signal) and amplitude (of the high-frequency signal). We do so here in one way, by computing the phase-amplitude plot.

##The phase-amplitude plot

To start, define the two-column phase-amplitude vector, where 𝜙(𝑖)
 is the phase of the low-frequency band activity at time index 𝑖
, and 𝐴(𝑖)
 is the amplitude of the high-frequency band activity at time index 𝑖
. In other words, each row defines the instantaneous phase and amplitude of the low- and high-frequency filtered data, respectively.

We now use this two-column vector to assess whether the phase and amplitude envelope are related. Let’s begin by plotting the average amplitude versus phase. We divide the phase interval into bins of size 0.1 beginning at −𝜋
 and ending at 𝜋.
 the choice of bin size is somewhat arbitrary; this choice will work fine, but you might consider the impact of other choices.

In [ ]:
p_bins = np.arange(-np.pi, np.pi, 0.1)
a_mean = np.zeros(np.size(p_bins)-1)
p_mean = np.zeros(np.size(p_bins)-1)
for k in range(np.size(p_bins)-1):     #For each phase bin,
    pL = p_bins[k]                  #... lower phase limit,
    pR = p_bins[k+1]                #... upper phase limit.
    indices=(phi>=pL) & (phi<pR)    #Find phases falling in bin,
    a_mean[k] = np.mean(amp[indices])  #... compute mean amplitude,
    p_mean[k] = np.mean([pL, pR])      #... save center phase.
plt.plot(p_mean, a_mean)                #Plot the phase versus amplitude,
plt.ylabel('High-frequency amplitude')  #... with axes labeled.
plt.xlabel('Low-frequency phase');

Question: Consider this plot of the average amplitude versus phase. Does this result suggest CFC occurs in these data? If there were no CFC occurring in the data, what would you expect this plot to look like?

><b>Task:</b> Re-make the above graph as a bar graph instead of a line plot.

As a basic statistic to capture the extent of this relation, we compute the difference between the maximum and minimum of the average amplitude envelope over phases. Let’s assign this difference the label ℎ
. In Python,

In [ ]:
h = max(a_mean)-min(a_mean)
print(h)

This value, on its own, is difficult to interpret. Is it bigger or smaller than we expect? To assess the significance of ℎ
, let’s generate a surrogate phase-amplitude vector by resampling without replacement the amplitude time series (i.e., the second column of the phase-amplitude vector). By performing this resampling, we reassign each phase an amplitude chosen randomly from the entire 100 s LFP recording. We expect that if CFC does exist in these data, then the timing of the phase and amplitude vectors will be important; for CFC to occur, the amplitude and phase must coordinate in time. By disrupting this timing in the resampling procedure, we expect to eliminate the coordination between amplitude and phase necessary to produce CFC.

For each surrogate phase-amplitude vector, we compute the statistic ℎ
. To generate a distribution of ℎ
 values, let’s repeat 1,000 times this process of creating surrogate data through resampling and computing ℎ
.

In [ ]:
n_surrogates = 1000;                    #Define no. of surrogates.
hS = np.zeros(n_surrogates)                #Vector to hold h results.
for ns in range(n_surrogates):          #For each surrogate,
    ampS = amp[np.random.randint(0,N,N)]          #Resample amplitude,
    p_bins = np.arange(-np.pi, np.pi, 0.1)       #Define the phase bins
    a_mean = np.zeros(np.size(p_bins)-1)      #Vector for average amps.
    p_mean = np.zeros(np.size(p_bins)-1)      #Vector for phase bins.
    for k in range(np.size(p_bins)-1):
        pL = p_bins[k]                  #... lower phase limit,
        pR = p_bins[k+1]                #... upper phase limit.
        indices=(phi>=pL) & (phi<pR)    #Find phases falling in bin,
        a_mean[k] = np.mean(ampS[indices]) #... compute mean amplitude,
        p_mean[k] = np.mean([pL, pR])      #... save center phase.
    hS[ns] = max(a_mean)-min(a_mean)    # Store surrogate h.

In this code, we first define the number of surrogates (variable `n_surrogates`) and a vector to store the statistic ℎ
 computed for each surrogate phase-amplitude vector (variable `hS`). Then, for each surrogate, we use the Python function `randint` to randomly permute the amplitude vector without replacement. We then use this permuted amplitude vector (variable `ampS`) and the original phase vector (variable `phi`) to compute ℎ
; this last step utilizes the Python code developed earlier to compute `h` for the original (unpermuted) data.

Let’s now view the results of this resampling procedure by creating a histogram of the variable `hS`, and compare this distribution to the value of `h` we computed earlier.

In [ ]:
counts, _, _ = plt.hist(hS, label='surrogates')               # Plot the histogram of hS, and save the bin counts.
plt.vlines(h, 0, max(counts), colors='red', label='h', lw=2)  # Plot the observed h,
plt.legend();  

The value of ℎ
 computed from the original data (`h`) lies far outside the distribution of surrogate ℎ
 values (`hS`). To compute a 𝑝
-value, we determine the proportion of surrogate ℎ
 values greater than the observed ℎ
 value:

In [ ]:
p = sum([s > h for s in hS]) / len(hS)
print(p)

We find a 𝑝
-value that is very small; there are no surrogate values of ℎ
 that exceed the observed value of ℎ
. We therefore conclude that in this case the observed value of ℎ
 is significant. As a statistician would say, we reject the null hypothesis of no CFC between the phase of the 5-7 Hz low-frequency band and the amplitude of the 80-120 Hz high frequency band.